
required tmc data : TMC_Identification.csv

In [127]:
import os
import math
import datetime
import numpy as np
import pandas as pd

import time
import os.path
import MapMatching4GMNS

In [128]:
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [129]:
def Create_Boundary(node_csv):
    
    minx = node_csv['x_coord'].min()
    miny = node_csv['y_coord'].min()
    maxx = node_csv['x_coord'].max()
    maxy = node_csv['y_coord'].max()
    bbox = [minx,miny,maxx,maxy]
    return bbox

In [133]:
def Convert_TMC(tmc_path,gmns_path):
    create_folder(gmns_path)
    print('reading tmc data...')

    tmc = pd.read_csv(tmc_path + os.sep + 'TMC_Identification.csv')

    '''build node.csv'''
    print('converting tmc data into gmns format...')


    node_csv = pd.DataFrame()
    node_csv['name'] = None
    node_csv['x_coord'] = None
    node_csv['y_coord'] = None
    node_csv['z_coord'] = None
    node_csv['node_type'] = None
    node_csv['ctrl_type'] = None
    node_csv['zone_id'] = None
    node_csv['parent_node_id'] = None
    node_csv['geometry'] = None

    for i in range(0,len(tmc)-1):
        if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
            node_csv = node_csv.append({'name': tmc.loc[i,'tmc'],\
                                        'x_coord': tmc.loc[i,'x_coord'], \
                                        'y_coord': tmc.loc[i,'y_coord'],\
                                        'z_coord': None,\
                                        'node_type': 'tmc_start',\
                                        'ctrl_type': None,\
                                        'zone_id': None,\
                                        'parent_node_id': None,\
                                        'geometry': "POINT (" + tmc.loc[i,'x_coord'].astype(str) + " " + tmc.loc[i,'y_coord'].astype(str) +")"}, ignore_index=True)
        else:
            node_csv = node_csv.append({'name': tmc.loc[i,'tmc'],\
                                        'x_coord': tmc.loc[i,'x_coord'], \
                                        'y_coord': tmc.loc[i,'y_coord'],\
                                        'z_coord': None,\
                                        'node_type': 'tmc_start',\
                                        'ctrl_type': None,\
                                        'zone_id': None,\
                                        'parent_node_id': None,\
                                        'geometry': "POINT (" + tmc.loc[i,'x_coord'].astype(str) + " " + tmc.loc[i,'y_coord'].astype(str) +")"}, ignore_index=True)
            node_csv = node_csv.append({'name': tmc.loc[i,'tmc']+'END',\
                                        'x_coord': tmc.loc[i,'end_longitude'], \
                                        'y_coord': tmc.loc[i,'end_latitude'],\
                                        'z_coord': None,\
                                        'node_type': 'tmc_end',\
                                        'ctrl_type': None,\
                                        'zone_id': None,\
                                        'parent_node_id': None,\
                                        'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)


    node_csv = node_csv.append({'name': tmc.loc[i+1,'tmc'],\
                                        'x_coord': tmc.loc[i+1,'x_coord'], \
                                        'y_coord': tmc.loc[i+1,'y_coord'],\
                                        'z_coord': None,\
                                        'node_type': 'tmc_start',\
                                        'ctrl_type': None,\
                                        'zone_id': None,\
                                        'parent_node_id': None,\
                                        'geometry': "POINT (" + tmc.loc[i+1,'x_coord'].astype(str) + " " + tmc.loc[i+1,'y_coord'].astype(str) +")"}, ignore_index=True)

    node_csv = node_csv.append({'name': tmc.loc[i+1,'tmc']+'END',\
                                        'x_coord': tmc.loc[i+1,'end_longitude'], \
                                        'y_coord': tmc.loc[i+1,'end_latitude'],\
                                        'z_coord': None,\
                                        'node_type': 'tmc_end',\
                                        'ctrl_type': None,\
                                        'zone_id': None,\
                                        'parent_node_id': None,\
                                        'geometry': "POINT (" + tmc.loc[i+1,'end_longitude'].astype(str) + " " + tmc.loc[i+1,'end_latitude'].astype(str) +")"}, ignore_index=True)

    node_csv.index.name = 'node_id'

    node_csv.index += 100000001 #index from 0

    node_csv.to_csv(gmns_path + os.sep + '/node_tmc.csv')


    '''build link.csv'''
    link_csv = pd.DataFrame()
    link_csv['name'] = None
    link_csv['from_node_id'] = None
    link_csv['to_node_id'] = None
    link_csv['directed'] = None
    link_csv['geometry_id'] = None
    link_csv['geometry'] = None
    link_csv['dir_flag'] = None
    link_csv['parent_link_id'] = None
    link_csv['length'] = None
    link_csv['grade'] = None
    link_csv['facility_type'] = None
    link_csv['capacity'] = None
    link_csv['free_speed'] = None
    link_csv['lanes'] = None

    for i in range(0,len(tmc)):
        link_csv = link_csv.append({'name': tmc.loc[i,'tmc'],\
                                        'from_node_id': node_csv[(node_csv['x_coord']==tmc.loc[i,'x_coord']) & (node_csv['y_coord']==tmc.loc[i,'y_coord'])].index.values[0], \
                                        'to_node_id': node_csv[(node_csv['x_coord']==tmc.loc[i,'end_longitude']) & (node_csv['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                        'directed': 1,\
                                        'geometry_id': None,\
                                        'geometry': "LINESTRING (" + tmc.loc[i,'x_coord'].astype(str) + " " + tmc.loc[i,'y_coord'].astype(str) + "," +\
                                            tmc.loc[i,'end_longitude'].astype(str) + tmc.loc[i,'end_latitude'].astype(str) + ")",\
                                        'dir_flag': 1,\
                                        'parent_link_id': None,\
                                        'length': tmc.loc[i,'miles'],\
                                        'grade': None,\
                                        'facility_type': 'interstate' if tmc.loc[i,'road'][0] == 'I'else None ,\
                                        'capacity':None,\
                                        'free_speed':None,\
                                        'lanes': None}, ignore_index=True)
    link_csv.index.name = 'link_id'
    link_csv.index += 100000001

    link_csv.to_csv(gmns_path + os.sep + '/link_tmc.csv')

    return node_csv, link_csv

In [134]:
"Step 0.1 Convert TMC Data into GMNS Format"

tmc_path = '/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/TMC'
gmns_path = '/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/GMNS'

# import tmc2gmns as gg
# node_transit,link_transit = gg.Convert_TMC(tmc_path,gmns_path)


In [135]:
"Step 0.2 Get the OSM Network"

import osm2gmns as og

osm_path = 'OSM/consolidated'

# net = og.getNetFromOSMFile('OSM/tmc_VA.osm',network_type=('auto'), default_lanes=True, default_speed=True)
# og.consolidateComplexIntersections(net)
# og.outputNetToCSV(net, output_folder=osm_path)



In [132]:
"Step 1 Create the Connector"

import tmc2gmns as gg
node,link_osm_connector = gg.CreatConnector_osm_tmc(osm_path,gmns_path)

ModuleNotFoundError: No module named 'tmc2gmns'

In [136]:
create_folder(gmns_path)
print('reading tmc data...')

tmc = pd.read_csv(tmc_path + os.sep + 'TMC_Identification_VA.csv')
# tmc = tmc.sort_values(by=['road_order'])

'''build node.csv'''
print('converting tmc data into gmns format...')


node_csv = pd.DataFrame()
node_csv['name'] = None
node_csv['x_coord'] = None
node_csv['y_coord'] = None
node_csv['z_coord'] = None
node_csv['node_type'] = None
node_csv['ctrl_type'] = None
node_csv['zone_id'] = None
node_csv['parent_node_id'] = None
node_csv['geometry'] = None

for i in range(0,len(tmc)-1):
    if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
        node_csv = node_csv.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
    else:
        node_csv = node_csv.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
        node_csv = node_csv.append({'name': tmc.loc[i,'tmc']+'END',\
                                    'x_coord': tmc.loc[i,'end_longitude'], \
                                    'y_coord': tmc.loc[i,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)


node_csv = node_csv.append({'name': tmc.loc[i+1,'tmc'],\
                                    'x_coord': tmc.loc[i+1,'start_longitude'], \
                                    'y_coord': tmc.loc[i+1,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'start_longitude'].astype(str) + " " + tmc.loc[i+1,'start_latitude'].astype(str) +")"}, ignore_index=True)

node_csv = node_csv.append({'name': tmc.loc[i+1,'tmc']+'END',\
                                    'x_coord': tmc.loc[i+1,'end_longitude'], \
                                    'y_coord': tmc.loc[i+1,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'end_longitude'].astype(str) + " " + tmc.loc[i+1,'end_latitude'].astype(str) +")"}, ignore_index=True)

node_csv.index.name = 'node_id'

node_csv.index += 100000001 #index from 0

node_csv.to_csv(gmns_path + os.sep + '/node_tmc.csv')

reading tmc data...
converting tmc data into gmns format...


In [137]:
'''build link.csv'''
link_csv = pd.DataFrame()
link_csv['name'] = None
link_csv['corridor_id'] = None
link_csv['corridor_link_order'] = None
link_csv['from_node_id'] = None
link_csv['to_node_id'] = None
link_csv['directed'] = None
link_csv['geometry_id'] = None
link_csv['geometry'] = None
link_csv['dir_flag'] = None
link_csv['parent_link_id'] = None
link_csv['length'] = None
link_csv['grade'] = None
link_csv['facility_type'] = None
link_csv['capacity'] = None
link_csv['free_speed'] = None
link_csv['lanes'] = None

for i in range(0,len(tmc)):
    link_csv = link_csv.append({'name': tmc.loc[i,'tmc'],\
                                'corridor_id': tmc.loc[i,'road']+'_'+tmc.loc[i,'direction'],\
                                'corridor_link_order' : tmc.loc[i,'road_order'],\
                                'from_node_id': node_csv[(node_csv['x_coord']==tmc.loc[i,'start_longitude']) & (node_csv['y_coord']==tmc.loc[i,'start_latitude'])].index.values[0], \
                                'to_node_id': node_csv[(node_csv['x_coord']==tmc.loc[i,'end_longitude']) & (node_csv['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                'directed': 1,\
                                'geometry_id': None,\
                                'geometry': "LINESTRING (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) + "," +\
                                    tmc.loc[i,'end_longitude'].astype(str) +" "+ tmc.loc[i,'end_latitude'].astype(str) + ")",\
                                'dir_flag': 1,\
                                'parent_link_id': None,\
                                'length': tmc.loc[i,'miles'],\
                                'grade': None,\
                                'facility_type': 'interstate' if tmc.loc[i,'road'][0] == 'I'else None ,\
                                'capacity':None,\
                                'free_speed':None,\
                                'lanes': None}, ignore_index=True)
link_csv.index.name = 'link_id'
link_csv.index += 100000001


link_csv.to_csv(gmns_path + os.sep + '/link_tmc.csv')


In [9]:
link_csv

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,parent_link_id,length,grade,facility_type,capacity,free_speed,lanes
link_id,,,,,,,,,,,,,,,,
100000001,110N05711,VA-7_EASTBOUND,38.0,100000001,100000002,1,None,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",1,None,0.038620,None,None,None,None,None
100000002,110-05710,VA-7_EASTBOUND,39.0,100000002,100000003,1,None,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",1,None,0.164895,None,None,None,None,None
100000003,110N05710,VA-7_EASTBOUND,40.0,100000003,100000004,1,None,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",1,None,0.512158,None,None,None,None,None
100000004,110-05709,VA-7_EASTBOUND,41.0,100000004,100000005,1,None,"LINESTRING (-77.3605 39.0086,-77.3502 39.0028)",1,None,0.682589,None,None,None,None,None
100000005,110-05708,VA-7_EASTBOUND,42.0,100000005,100000006,1,None,"LINESTRING (-77.3502 39.0028,-77.3455 39.0005)",1,None,0.296731,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000058,110+05709,VA-7_WESTBOUND,45.0,100000059,100000060,1,None,"LINESTRING (-77.346 39.001,-77.35 39.003)",1,None,0.254799,None,None,None,None,None
100000059,110+05710,VA-7_WESTBOUND,46.0,100000060,100000061,1,None,"LINESTRING (-77.35 39.003,-77.3609 39.009)",1,None,0.718593,None,None,None,None,None
100000060,110P05710,VA-7_WESTBOUND,47.0,100000061,100000062,1,None,"LINESTRING (-77.3609 39.009,-77.3673 39.0125)",1,None,0.421999,None,None,None,None,None


In [10]:
tmc

,tmc,road,direction,intersection,state,county,zip,start_latitude,start_longitude,end_latitude,end_longitude,miles,road_order
0,110N05711,VA-7,EASTBOUND,VA-228/DRANESVILLE RD,VA,FAIRFAX COUNTY,20170,39.0144,-77.3715,39.0141,-77.3709,0.038620,38.0
1,110-05710,VA-7,EASTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0141,-77.3709,39.0127,-77.3684,0.164895,39.0
2,110N05710,VA-7,EASTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0127,-77.3684,39.0086,-77.3605,0.512158,40.0
3,110-05709,VA-7,EASTBOUND,SUGARLAND RD,VA,FAIRFAX COUNTY,20170,39.0086,-77.3605,39.0028,-77.3502,0.682589,41.0
4,110-05708,VA-7,EASTBOUND,VA-193/GEORGETOWN PIKE,VA,FAIRFAX COUNTY,20170,39.0028,-77.3502,39.0005,-77.3455,0.296731,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,110+05709,VA-7,WESTBOUND,SUGARLAND RD,VA,FAIRFAX COUNTY,20170,39.0010,-77.3460,39.0030,-77.3500,0.254799,45.0
58,110+05710,VA-7,WESTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0030,-77.3500,39.0090,-77.3609,0.718593,46.0
59,110P05710,VA-7,WESTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0090,-77.3609,39.0125,-77.3673,0.421999,47.0
60,110+05711,VA-7,WESTBOUND,VA-228/DRANESVILLE RD,VA,FAIRFAX COUNTY,20170,39.0125,-77.3673,39.0143,-77.3707,0.220479,48.0


In [11]:
'''build link.csv'''
node_csv


,name,x_coord,y_coord,z_coord,node_type,ctrl_type,zone_id,parent_node_id,geometry
node_id,,,,,,,,,
100000001,110N05711,-77.3715,39.0144,None,tmc_start,None,None,None,POINT (-77.3715 39.0144)
100000002,110-05710,-77.3709,39.0141,None,tmc_start,None,None,None,POINT (-77.3709 39.0141)
100000003,110N05710,-77.3684,39.0127,None,tmc_start,None,None,None,POINT (-77.3684 39.0127)
100000004,110-05709,-77.3605,39.0086,None,tmc_start,None,None,None,POINT (-77.3605 39.0086)
100000005,110-05708,-77.3502,39.0028,None,tmc_start,None,None,None,POINT (-77.3502 39.0028)
...,...,...,...,...,...,...,...,...,...
100000060,110+05710,-77.3500,39.0030,None,tmc_start,None,None,None,POINT (-77.35 39.003)
100000061,110P05710,-77.3609,39.0090,None,tmc_start,None,None,None,POINT (-77.3609 39.009)
100000062,110+05711,-77.3673,39.0125,None,tmc_start,None,None,None,POINT (-77.3673 39.0125)


In [138]:
import os
import math
import heapq
import numpy as np
import pandas as pd


# WGS84 transfer coordinate system to distance: meter
def LLs2Dist(lon1, lat1, lon2, lat2):
    R = 6371
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
    a = math.sin(dLat / 2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180.0) * math.cos(lat2 * math.pi / 180.0) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    dist = R * c * 1000
    return dist


def createConnector(dataList,from_node_id, to_node_id):
    link = []
    from_node_id_x = dataList[from_node_id]['x_coord']
    from_node_id_y = dataList[from_node_id]['y_coord']
    to_node_id_x = dataList[to_node_id]['x_coord']
    to_node_id_y = dataList[to_node_id]['y_coord']
    length = LLs2Dist(from_node_id_x,from_node_id_y,to_node_id_x,to_node_id_y)
    geometry = 'LINESTRING (' + str(from_node_id_x)+' '+str(from_node_id_y)+', '+str(to_node_id_x)+' '+str(to_node_id_y)+')'
        
    link = [from_node_id, to_node_id, length, geometry]
    return link




print('creating connector between osm network and tmc data...')

node_tmc = pd.read_csv(gmns_path + os.sep +'/node_tmc.csv',low_memory=False) # transit node
node_road = pd.read_csv(osm_path + os.sep +'/node.csv', encoding='gbk',low_memory=False)

node_combine = pd.DataFrame()
node_combine['node_id']= node_road['node_id'].tolist() + node_tmc['node_id'].tolist()
node_combine['name']= node_road['name'].tolist() + node_tmc['name'].tolist()
node_combine['x_coord']= node_road['x_coord'].tolist() + node_tmc['x_coord'].tolist()
node_combine['y_coord'] = node_road['y_coord'].tolist() + node_tmc['y_coord'].tolist()
node_combine['node_type']= node_road['node_type'].tolist() + node_tmc['node_type'].tolist()


node_combine.to_csv(gmns_path + os.sep +'/node_combine.csv', index=False)
    
link_road = pd.read_csv(osm_path + os.sep +'/link.csv', encoding='gbk',low_memory=False)
link_road = link_road.drop(['link_id', 'osm_way_id', 'from_biway'], axis=1)
    
dataList_tmc = {}
gp = node_tmc.groupby('node_id')
for key, form in gp:
    dataList_tmc[key] = {
        'x_coord': form['x_coord'].values[0],
        'y_coord': form['y_coord'].values[0]
        }

dataList_road = {}
gp = node_road.groupby('node_id')
for key, form in gp:
    dataList_road[key] = {
        'x_coord': form['x_coord'].values[0],
        'y_coord': form['y_coord'].values[0]
        }

coord_list = []
for key in dataList_road.keys(): 
    coord_list.append((dataList_road[key]['x_coord'],dataList_road[key]['y_coord']))
coord_array = np.array(coord_list)


dataList = {}  #road node+tmc node
gp = node_combine.groupby('node_id')
for key, form in gp:
    dataList[key] = {
        'x_coord': form['x_coord'].values[0],
        'y_coord': form['y_coord'].values[0]
        }

# print('building connector...')
link_list = []

for key in dataList_tmc.keys(): 
    coord = np.array((dataList_tmc[key]['x_coord'],dataList_tmc[key]['y_coord']))
    coord_diff = coord_array - coord
    coord_diff_square = np.power(coord_diff,2)
    coord_diff_sum_square = coord_diff_square.sum(axis=1)
    distance = np.sqrt(coord_diff_sum_square)  #distance array between one tmc node to all road node 
    
    count = 1
    while (count):
        idx_temp = heapq.nsmallest(count, distance.tolist())
        idx = np.where(distance == idx_temp[count-1])  #index of the nearest road node
        active_node = node_road['node_id'].iloc[idx[0]]  #id of the nearest road node
        if ((active_node[idx[0][0]] in link_road['from_node_id'].tolist()) and (active_node[idx[0][0]] in link_road['to_node_id'].tolist())): #check the road node is the from/to node of road link 
            active_link1 = createConnector(dataList,active_node[idx[0][0]], key) #create link between the nearest road node and the selected tmc node
            active_link2= createConnector(dataList,key, active_node[idx[0][0]])
            link_list.append(active_link1)
            link_list.append(active_link2)
            break
        else:
            count += 1


connector_csv = pd.DataFrame()
connector_csv = pd.DataFrame(link_list, columns=['from_node_id','to_node_id','length','geometry']).drop_duplicates()    


connector_csv['name'] = None
connector_csv['link_type'] = None
connector_csv['link_type_name'] = 'connector'
connector_csv['dir_flag'] = 1
connector_csv['lanes'] = 1
connector_csv['free_speed'] = None
connector_csv['capacity'] = None
connector_csv['allowed_uses'] = None



combined_link = pd.concat([link_road,connector_csv], axis=0, ignore_index=True)

combined_link.index.name = 'link_id'
combined_link.index += 1

combined_link.to_csv(gmns_path + os.sep +'/link_osm_connector.csv')



creating connector between osm network and tmc data...


In [13]:
node_road

,name,node_id,osm_node_id,osm_highway,zone_id,ctrl_type,node_type,activity_type,is_boundary,x_coord,y_coord,main_node_id,poi_id,notes
0,NaN,0,288155,NaN,NaN,0,NaN,NaN,NaN,-77.267406,38.878836,NaN,NaN,NaN
1,NaN,1,288163,NaN,NaN,0,NaN,NaN,NaN,-77.265194,38.875829,NaN,NaN,NaN
2,NaN,2,49130347,NaN,NaN,0,NaN,NaN,NaN,-77.190795,38.996796,NaN,NaN,NaN
3,NaN,3,49131276,NaN,NaN,0,NaN,NaN,NaN,-77.184717,38.973858,NaN,NaN,NaN
4,NaN,4,49132520,NaN,NaN,0,NaN,NaN,NaN,-77.187825,38.974418,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14457,NaN,14933,1839494421;1839510595,traffic_signals,NaN,1,NaN,NaN,NaN,-77.369253,38.880889,183.0,NaN,NaN
14458,NaN,14934,1840601958;1840601988;1840602023,traffic_signals,NaN,1,NaN,NaN,NaN,-77.368589,38.950817,177.0,NaN,NaN
14459,NaN,14935,1843418021;1843418031,traffic_signals,NaN,1,NaN,NaN,NaN,-77.341451,38.971025,86.0,NaN,NaN
14460,NaN,14936,3746828277;5534645925,traffic_signals,NaN,1,NaN,NaN,NaN,-77.184447,38.890839,110.0,NaN,NaN


In [14]:
combined_link

,name,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses
link_id,,,,,,,,,,,,
1,NaN,4,5,1,146.766935,1,29,NaN,unclassified,11,"LINESTRING (-77.1878249 38.9744185, -77.187663...",auto
2,NaN,5,4,1,146.766935,1,29,NaN,unclassified,11,"LINESTRING (-77.1869767 38.9755412, -77.187013...",auto
3,NaN,6,7,1,106.872460,1,29,NaN,unclassified,11,"LINESTRING (-77.194886 38.9751513, -77.1950799...",auto
4,NaN,7,6,1,106.872460,1,29,NaN,unclassified,11,"LINESTRING (-77.1960675 38.9754343, -77.196016...",auto
5,NaN,7,8,1,85.661050,1,29,NaN,unclassified,11,"LINESTRING (-77.1960675 38.9754343, -77.196222...",auto
...,...,...,...,...,...,...,...,...,...,...,...,...
30827,None,100000062,12388,1,84.064216,1,None,None,connector,None,"LINESTRING (-77.3673 39.0125, -77.3680805 39.0...",None
30828,None,10238,100000063,1,7.938151,1,None,None,connector,None,"LINESTRING (-77.3707839 39.014329100000005, -7...",None
30829,None,100000063,10238,1,7.938151,1,None,None,connector,None,"LINESTRING (-77.3707 39.0143, -77.3707839 39.0...",None


In [62]:
reading = pd.read_csv(tmc_path + os.sep + 'Reading_VA.csv')
reading = reading[pd.to_datetime(reading['measurement_tstamp'], format='%Y-%m-%d %H:%M:%S')<datetime.datetime.strptime('2015-04-01 02:00:00', '%Y-%m-%d %H:%M:%S')]
reading

,_vatmc_code,measurement_tstamp,speed,reference_speed
0,110-05695,2015-04-01 00:00:00,31.0,31.0
1,110+05696,2015-04-01 00:00:00,32.0,32.0
2,110P05699,2015-04-01 00:00:00,36.0,36.0
3,110+05704,2015-04-01 00:00:00,50.0,50.0
4,110+05695,2015-04-01 00:00:00,31.0,31.0
...,...,...,...,...
1411,110-05709,2015-04-01 01:55:00,48.0,48.0
1412,110N05695,2015-04-01 01:55:00,33.0,33.0
1413,110-05707,2015-04-01 01:55:00,50.0,50.0
1414,110+05700,2015-04-01 01:55:00,40.0,40.0


In [105]:
link_performance_csv = pd.DataFrame()
link_performance_csv['name'] = None
link_performance_csv['corridor_id'] = None
link_performance_csv['corridor_link_order'] = None
link_performance_csv['from_node_id'] = None
link_performance_csv['to_node_id'] = None
link_performance_csv['timestamp'] = None
link_performance_csv['volume'] = None
link_performance_csv['travel_time'] = None
link_performance_csv['speed'] = None
link_performance_csv['reference_speed'] = None
link_performance_csv['density'] = None
link_performance_csv['queue'] = None
link_performance_csv['notes'] = None


gp = reading.groupby('measurement_tstamp')
for key, form in gp:
    # print(key)
    for i in link_csv.index:
        form_selected = form[form['_vatmc_code']==link_csv['name'][i]]
        if len(form_selected)>0:
        # break
            link_performance_csv = link_performance_csv.append({'name': link_csv['name'][i],\
                                            'corridor_id': link_csv['corridor_id'][i],\
                                            'corridor_link_order' : link_csv['corridor_link_order'][i],\
                                            'from_node_id': link_csv.loc[i,'from_node_id'], \
                                            'to_node_id': link_csv.loc[i,'to_node_id'], \
                                            'timestamp': form_selected['measurement_tstamp'].values[0][0:10]+'T'+form_selected['measurement_tstamp'].values[0][11:13]+':'+form_selected['measurement_tstamp'].values[0][14:16],\
                                            'volume': None,\
                                            'travel_time': link_csv['length'][i]/form_selected['speed'].values[0],\
                                            'speed': form_selected['speed'].values[0],\
                                            'reference_speed': form_selected['reference_speed'].values[0],\
                                            'density': None,\
                                            'queue': None,\
                                            'notes': None }, ignore_index=True)
        else:
            link_performance_csv = link_performance_csv.append({'name': link_csv['name'][i],\
                                            'corridor_id': link_csv['corridor_id'][i],\
                                            'corridor_link_order' : link_csv['corridor_link_order'][i],\
                                            'from_node_id': link_csv.loc[i,'from_node_id'], \
                                            'to_node_id': link_csv.loc[i,'to_node_id'], \
                                            'timestamp': None,\
                                            'volume': None,\
                                            'travel_time': None,\
                                            'speed': None,\
                                            'reference_speed': None,\
                                            'density': None,\
                                            'queue': None,\
                                            'notes': None }, ignore_index=True)

link_performance_csv.to_csv(tmc_path + os.sep +'/link_performance_tmc.csv',index = False)


In [ ]:
# link_performance_csv = pd.DataFrame()
# link_performance_csv['corridor_id'] = None
# link_performance_csv['corridor_link_order'] = None
# link_performance_csv['from_node_id'] = None
# link_performance_csv['to_node_id'] = None
# link_performance_csv['timestamp'] = None
# link_performance_csv['volume'] = None
# link_performance_csv['travel_time'] = None
# link_performance_csv['speed'] = None
# link_performance_csv['reference_speed'] = None
# link_performance_csv['density'] = None
# link_performance_csv['queue'] = None
# link_performance_csv['notes'] = None


# gp = reading.groupby('measurement_tstamp')
# for key, form in gp:
#     # print(key)
#     for i in link_csv.index:
#         form_selected = form[form['_vatmc_code']==link_csv['name'][i]]
#         if len(form_selected)>0:
#         # break
#             link_performance_csv = link_performance_csv.append({'corridor_id': link_csv['name'][i]+'_'+tmc.loc[tmc['tmc']==link_csv['name'][i],'direction'].values[0],\
#                                             'corridor_link_order' : tmc.loc[tmc['tmc']==link_csv['name'][i],'road_order'].values[0],\
#                                             'from_node_id': link_csv.loc[i,'from_node_id'], \
#                                             'to_node_id': link_csv.loc[i,'to_node_id'], \
#                                             'timestamp': form_selected['measurement_tstamp'].values[0][0:10]+'T'+form_selected['measurement_tstamp'].values[0][11:13]+':'+form_selected['measurement_tstamp'].values[0][14:16],\
#                                             'volume': None,\
#                                             'travel_time': link_csv['length'][i]/form_selected['speed'].values[0],\
#                                             'speed': form_selected['speed'].values[0],\
#                                             'reference_speed': form_selected['reference_speed'].values[0],\
#                                             'density': None,\
#                                             'queue': None,\
#                                             'notes': None }, ignore_index=True)
#         else:
#             link_performance_csv = link_performance_csv.append({'corridor_id': link_csv['name'][i]+'_'+tmc.loc[tmc['tmc']==link_csv['name'][i],'direction'].values[0],\
#                                             'corridor_link_order' : tmc.loc[tmc['tmc']==link_csv['name'][i],'road_order'].values[0],\
#                                             'from_node_id': link_csv.loc[i,'from_node_id'], \
#                                             'to_node_id': link_csv.loc[i,'to_node_id'], \
#                                             'timestamp': None,\
#                                             'volume': None,\
#                                             'travel_time': None,\
#                                             'speed': None,\
#                                             'reference_speed': None,\
#                                             'density': None,\
#                                             'queue': None,\
#                                             'notes': None }, ignore_index=True)

# link_performance_csv.to_csv(tmc_path + os.sep +'/link_performance_tmc.csv',index = False)


In [17]:
link_performance_csv

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,timestamp,volume,travel_time,speed,reference_speed,density,queue,notes
0,110N05711,VA-7_EASTBOUND,38.0,100000001,100000002,2015-04-01T00:00,None,0.000877727,44,44,None,None,None
1,110-05710,VA-7_EASTBOUND,39.0,100000002,100000003,2015-04-01T00:00,None,0.00374761,44,44,None,None,None
2,110N05710,VA-7_EASTBOUND,40.0,100000003,100000004,2015-04-01T00:00,None,0.0104522,49,49,None,None,None
3,110-05709,VA-7_EASTBOUND,41.0,100000004,100000005,2015-04-01T00:00,None,0.0142206,48,48,None,None,None
4,110-05708,VA-7_EASTBOUND,42.0,100000005,100000006,2015-04-01T00:00,None,0.0061819,48,48,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5947,110+05709,VA-7_WESTBOUND,45.0,100000059,100000060,2015-04-01T07:55,None,0.00636997,40,47,None,None,None
5948,110+05710,VA-7_WESTBOUND,46.0,100000060,100000061,2015-04-01T07:55,None,0.0150997,47.59,48,None,None,None
5949,110P05710,VA-7_WESTBOUND,47.0,100000061,100000062,2015-04-01T07:55,None,0.00917389,46,49,None,None,None
5950,110+05711,VA-7_WESTBOUND,48.0,100000062,100000063,2015-04-01T07:55,None,0.0051963,42.43,41,None,None,None


In [17]:
tmc

,tmc,road,direction,intersection,state,county,zip,start_latitude,start_longitude,end_latitude,end_longitude,miles,road_order
0,110N05711,VA-7,EASTBOUND,VA-228/DRANESVILLE RD,VA,FAIRFAX COUNTY,20170,39.0144,-77.3715,39.0141,-77.3709,0.038620,38.0
1,110-05710,VA-7,EASTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0141,-77.3709,39.0127,-77.3684,0.164895,39.0
2,110N05710,VA-7,EASTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0127,-77.3684,39.0086,-77.3605,0.512158,40.0
3,110-05709,VA-7,EASTBOUND,SUGARLAND RD,VA,FAIRFAX COUNTY,20170,39.0086,-77.3605,39.0028,-77.3502,0.682589,41.0
4,110-05708,VA-7,EASTBOUND,VA-193/GEORGETOWN PIKE,VA,FAIRFAX COUNTY,20170,39.0028,-77.3502,39.0005,-77.3455,0.296731,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,110+05709,VA-7,WESTBOUND,SUGARLAND RD,VA,FAIRFAX COUNTY,20170,39.0010,-77.3460,39.0030,-77.3500,0.254799,45.0
58,110+05710,VA-7,WESTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0030,-77.3500,39.0090,-77.3609,0.718593,46.0
59,110P05710,VA-7,WESTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0090,-77.3609,39.0125,-77.3673,0.421999,47.0
60,110+05711,VA-7,WESTBOUND,VA-228/DRANESVILLE RD,VA,FAIRFAX COUNTY,20170,39.0125,-77.3673,39.0143,-77.3707,0.220479,48.0


In [18]:
'''numeric trace_id '''
trace_csv = pd.DataFrame()
trace_csv['corridor_id'] = None
trace_csv['agent_id'] = None
trace_csv['date'] = None
trace_csv['tmc'] = None
trace_csv['trace_id'] = None
trace_csv['hh'] = None
trace_csv['mm'] = None
trace_csv['ss'] = None
trace_csv['y_coord'] = None
trace_csv['x_coord'] = None

agent_id = 1
trace_id = 0

for i in range(0,len(tmc)-1):
    if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
        trace_csv = trace_csv.append({'corridor_id': tmc.loc[i,'road'] + '_' + tmc.loc[i,'direction'],\
                                    'agent_id': agent_id,\
                                    'date': None, \
                                    'tmc': tmc.loc[i,'tmc'],\
                                    'trace_id': trace_id,\
                                    'hh': None,\
                                    'mm': 'None',\
                                    'ss': None,\
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'x_coord': tmc.loc[i,'start_longitude']}, ignore_index=True)
        trace_id +=1
    else:
        trace_csv = trace_csv.append({'corridor_id': tmc.loc[i,'road'] + '_' + tmc.loc[i,'direction'],\
                                    'agent_id': agent_id,\
                                    'date': None, \
                                    'tmc': tmc.loc[i,'tmc'],\
                                    'trace_id': trace_id,\
                                    'hh': None,\
                                    'mm': 'None',\
                                    'ss': None,\
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'x_coord': tmc.loc[i,'start_longitude']}, ignore_index=True)
        trace_id += 1

        trace_csv = trace_csv.append({'corridor_id': tmc.loc[i,'road'] + '_' + tmc.loc[i,'direction'],\
                                    'agent_id': agent_id,\
                                    'date': None, \
                                    'tmc': tmc.loc[i,'tmc'],\
                                    'trace_id': trace_id,\
                                    'hh': None,\
                                    'mm': 'None',\
                                    'ss': None,\
                                    'y_coord': tmc.loc[i,'end_latitude'],\
                                    'x_coord': tmc.loc[i,'end_longitude']}, ignore_index=True)
        agent_id += 1
        trace_id = 0
        
trace_csv = trace_csv.append({'corridor_id': tmc.loc[i+1,'road'] + '_' + tmc.loc[i+1,'direction'],\
                                    'agent_id': agent_id,\
                                    'date': None, \
                                    'tmc': tmc.loc[i+1,'tmc'],\
                                    'trace_id': trace_id,\
                                    'hh': None,\
                                    'mm': 'None',\
                                    'ss': None,\
                                    'y_coord': tmc.loc[i+1,'start_latitude'],\
                                    'x_coord': tmc.loc[i+1,'start_longitude']}, ignore_index=True)
trace_id +=1
trace_csv = trace_csv.append({'corridor_id': tmc.loc[i+1,'road'] + '_' + tmc.loc[i+1,'direction'],\
                                    'agent_id': agent_id,\
                                    'date': None, \
                                    'tmc': tmc.loc[i+1,'tmc'],\
                                    'trace_id': trace_id,\
                                    'hh': None,\
                                    'mm': 'None',\
                                    'ss': None,\
                                    'y_coord': tmc.loc[i+1,'end_latitude'],\
                                    'x_coord': tmc.loc[i+1,'end_longitude']}, ignore_index=True)

trace_csv.to_csv(tmc_path + os.sep +'/trace.csv')

In [19]:
trace_csv

,corridor_id,agent_id,date,tmc,trace_id,hh,mm,ss,y_coord,x_coord
0,VA-7_EASTBOUND,1,None,110N05711,0,None,None,None,39.0144,-77.3715
1,VA-7_EASTBOUND,1,None,110-05710,1,None,None,None,39.0141,-77.3709
2,VA-7_EASTBOUND,1,None,110N05710,2,None,None,None,39.0127,-77.3684
3,VA-7_EASTBOUND,1,None,110-05709,3,None,None,None,39.0086,-77.3605
4,VA-7_EASTBOUND,1,None,110-05708,4,None,None,None,39.0028,-77.3502
...,...,...,...,...,...,...,...,...,...,...
59,VA-7_WESTBOUND,2,None,110+05710,27,None,None,None,39.0030,-77.3500
60,VA-7_WESTBOUND,2,None,110P05710,28,None,None,None,39.0090,-77.3609
61,VA-7_WESTBOUND,2,None,110+05711,29,None,None,None,39.0125,-77.3673
62,VA-7_WESTBOUND,2,None,110P05711,30,None,None,None,39.0143,-77.3707


In [72]:
# '''string tmc id'''
# trace_csv = pd.DataFrame()
# trace_csv['corridor_id'] = None
# trace_csv['date'] = None
# trace_csv['tmc'] = None
# trace_csv['hh'] = None
# trace_csv['mm'] = None
# trace_csv['ss'] = None
# trace_csv['y_coord'] = None
# trace_csv['x_coord'] = None

# # agent_id = 1
# # trace_id = 0

# for i in range(0,len(tmc)-1):
#     if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
#         trace_csv = trace_csv.append({'corridor_id': tmc.loc[i,'road'] + '_' + tmc.loc[i,'direction'],\
#                                     'date': None, \
#                                     'tmc': tmc.loc[i,'tmc'],\
#                                     'hh': None,\
#                                     'mm': 'None',\
#                                     'ss': None,\
#                                     'y_coord': tmc.loc[i,'start_latitude'],\
#                                     'x_coord': tmc.loc[i,'start_longitude']}, ignore_index=True)
#         # trace_id +=1
#     else:
#         trace_csv = trace_csv.append({'corridor_id': tmc.loc[i,'road'] + '_' + tmc.loc[i,'direction'],\
#                                     'date': None, \
#                                     'tmc': tmc.loc[i,'tmc'],\
#                                     'hh': None,\
#                                     'mm': 'None',\
#                                     'ss': None,\
#                                     'y_coord': tmc.loc[i,'start_latitude'],\
#                                     'x_coord': tmc.loc[i,'start_longitude']}, ignore_index=True)

#         trace_csv = trace_csv.append({'corridor_id': tmc.loc[i,'road'] + '_' + tmc.loc[i,'direction'],\
#                                     'date': None, \
#                                     'tmc': tmc.loc[i,'tmc'],\
#                                     'hh': None,\
#                                     'mm': 'None',\
#                                     'ss': None,\
#                                     'y_coord': tmc.loc[i,'end_latitude'],\
#                                     'x_coord': tmc.loc[i,'end_longitude']}, ignore_index=True)
#         # agent_id += 1
#         # trace_id = 0
        
# trace_csv = trace_csv.append({'corridor_id': tmc.loc[i+1,'road'] + '_' + tmc.loc[i+1,'direction'],\
#                                     'date': None, \
#                                     'tmc': tmc.loc[i+1,'tmc'],\
#                                     'hh': None,\
#                                     'mm': 'None',\
#                                     'ss': None,\
#                                     'y_coord': tmc.loc[i+1,'start_latitude'],\
#                                     'x_coord': tmc.loc[i+1,'start_longitude']}, ignore_index=True)
# # trace_id +=1
# trace_csv = trace_csv.append({'corridor_id': tmc.loc[i+1,'road'] + '_' + tmc.loc[i+1,'direction'],\
#                                     'date': None, \
#                                     'tmc': tmc.loc[i+1,'tmc'],\
#                                     'hh': None,\
#                                     'mm': 'None',\
#                                     'ss': None,\
#                                     'y_coord': tmc.loc[i+1,'end_latitude'],\
#                                     'x_coord': tmc.loc[i+1,'end_longitude']}, ignore_index=True)

# trace_csv.to_csv(tmc_path + os.sep +'/trace_csv.csv')

In [73]:
# trace_csv

,corridor_id,date,tmc,hh,mm,ss,y_coord,x_coord
0,VA-7_EASTBOUND,None,110N05711,None,None,None,39.0144,-77.3715
1,VA-7_EASTBOUND,None,110-05710,None,None,None,39.0141,-77.3709
2,VA-7_EASTBOUND,None,110N05710,None,None,None,39.0127,-77.3684
3,VA-7_EASTBOUND,None,110-05709,None,None,None,39.0086,-77.3605
4,VA-7_EASTBOUND,None,110-05708,None,None,None,39.0028,-77.3502
...,...,...,...,...,...,...,...,...
59,VA-7_WESTBOUND,None,110+05710,None,None,None,39.0030,-77.3500
60,VA-7_WESTBOUND,None,110P05710,None,None,None,39.0090,-77.3609
61,VA-7_WESTBOUND,None,110+05711,None,None,None,39.0125,-77.3673
62,VA-7_WESTBOUND,None,110P05711,None,None,None,39.0143,-77.3707


In [76]:
link_performance_csv

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,timestamp,volume,travel_time,speed,reference_speed,density,queue,notes
0,110N05711,VA-7_EASTBOUND,38.0,100000001,100000002,2015-04-01T00:00,NaN,0.000878,44.00,44.0,NaN,NaN,NaN
1,110-05710,VA-7_EASTBOUND,39.0,100000002,100000003,2015-04-01T00:00,NaN,0.003748,44.00,44.0,NaN,NaN,NaN
2,110N05710,VA-7_EASTBOUND,40.0,100000003,100000004,2015-04-01T00:00,NaN,0.010452,49.00,49.0,NaN,NaN,NaN
3,110-05709,VA-7_EASTBOUND,41.0,100000004,100000005,2015-04-01T00:00,NaN,0.014221,48.00,48.0,NaN,NaN,NaN
4,110-05708,VA-7_EASTBOUND,42.0,100000005,100000006,2015-04-01T00:00,NaN,0.006182,48.00,48.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5947,110+05709,VA-7_WESTBOUND,45.0,100000059,100000060,2015-04-01T07:55,NaN,0.006370,40.00,47.0,NaN,NaN,NaN
5948,110+05710,VA-7_WESTBOUND,46.0,100000060,100000061,2015-04-01T07:55,NaN,0.015100,47.59,48.0,NaN,NaN,NaN
5949,110P05710,VA-7_WESTBOUND,47.0,100000061,100000062,2015-04-01T07:55,NaN,0.009174,46.00,49.0,NaN,NaN,NaN
5950,110+05711,VA-7_WESTBOUND,48.0,100000062,100000063,2015-04-01T07:55,NaN,0.005196,42.43,41.0,NaN,NaN,NaN


In [20]:
input_agent_csv = pd.DataFrame()
input_agent_csv['agent_id'] = None
input_agent_csv['geometry'] = None
input_agent_csv['node_sequence'] = None

agent_id = 1
geometry_list = "LINESTRING ("
node_sequence_list = ''

for i in range(0,len(tmc)-1):
    if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
        geometry_list = geometry_list + tmc.loc[i,'start_longitude'].astype(str)+ " " + tmc.loc[i,'start_latitude'].astype(str) +','
        node_sequence_list = node_sequence_list + node_csv[(node_csv['x_coord'] ==  tmc.loc[i,'start_longitude'])&(node_csv['y_coord'] ==  tmc.loc[i,'start_latitude'])].index[0].astype(str) + ';'
        
    else:
        geometry_list = geometry_list + tmc.loc[i,'start_longitude'].astype(str)+ " " + tmc.loc[i,'start_latitude'].astype(str) +','
        node_sequence_list = node_sequence_list + node_csv[(node_csv['x_coord'] ==  tmc.loc[i,'start_longitude'])&(node_csv['y_coord'] ==  tmc.loc[i,'start_latitude'])].index[0].astype(str) + ';'


        geometry_list = geometry_list + tmc.loc[i,'end_longitude'].astype(str)+ " " + tmc.loc[i,'end_latitude'].astype(str) +',)'
        node_sequence_list = node_sequence_list + node_csv[(node_csv['x_coord'] ==  tmc.loc[i,'end_longitude'])&(node_csv['y_coord'] ==  tmc.loc[i,'end_latitude'])].index[0].astype(str) + ';'
        input_agent_csv = input_agent_csv.append({'agent_id': agent_id,\
                            'geometry': geometry_list,\
                                'node_sequence': node_sequence_list}, ignore_index=True)
        agent_id += 1
        geometry_list = "LINESTRING ("
        node_sequence_list = ''



geometry_list = geometry_list + tmc.loc[i+1,'start_longitude'].astype(str)+ " " + tmc.loc[i+1,'start_latitude'].astype(str) +','
node_sequence_list = node_sequence_list + node_csv[(node_csv['x_coord'] ==  tmc.loc[i+1,'start_longitude'])&(node_csv['y_coord'] ==  tmc.loc[i+1,'start_latitude'])].index[0].astype(str) + ';'


geometry_list = geometry_list + tmc.loc[i+1,'end_longitude'].astype(str)+ " " + tmc.loc[i+1,'end_latitude'].astype(str) +',)'
node_sequence_list = node_sequence_list + node_csv[(node_csv['x_coord'] ==  tmc.loc[i+1,'end_longitude'])&(node_csv['y_coord'] ==  tmc.loc[i+1,'end_latitude'])].index[0].astype(str) + ';'

input_agent_csv = input_agent_csv.append({'agent_id': agent_id,\
                            'geometry': geometry_list,\
                                'node_sequence': node_sequence_list}, ignore_index=True)
input_agent_csv.to_csv(tmc_path + os.sep +'/input_agent_csv.csv')

In [21]:
input_agent_csv

,agent_id,geometry,node_sequence
0,1,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141,...",100000001;100000002;100000003;100000004;100000...
1,2,"LINESTRING (-77.1943 38.8973,-77.1992 38.9005,...",100000033;100000034;100000035;100000036;100000...


In [22]:

# """install MapMatching4GMNS"""
!pip install MapMatching4GMNS --upgrade

distutils: /usr/local/home/ysx28/anaconda3/include/python3.8/UNKNOWN
sysconfig: /usr/local/home/ysx28/anaconda3/include/python3.8
user = False
home = None
root = None
prefix = None
distutils: /usr/local/home/ysx28/anaconda3/include/python3.8/UNKNOWN
sysconfig: /usr/local/home/ysx28/anaconda3/include/python3.8
user = False
home = None
root = None
prefix = None
You should consider upgrading via the '/usr/local/home/ysx28/anaconda3/bin/python3 -m pip install --upgrade pip' command.


In [23]:
if __name__ == "__main__":
    #If the output data exists, please remove.
    if(os.path.exists("agent.csv")):
       os.remove("agent.csv")
    if(os.path.exists("link_performance.csv")):
       os.remove("link_performance.csv")   
    
    start = time.time()
    MapMatching4GMNS.map_match()
    end = time.time()
    
    print('MapMatching4GMNS time cost: %.6f seconds' % (end - start))
    print("The output data is generated!")

call MapMatching4GMNS  dynamic library
\MapMatching4GMNS run starts
\MapMatching4GMNS run completes
MapMatching4GMNS time cost: 0.743150 seconds
The output data is generated!


In [121]:
link_road = pd.read_csv(osm_path + os.sep +'/link.csv', encoding='gbk',low_memory=False)
link_performance_tmc = pd.read_csv(tmc_path + os.sep +'/link_performance_tmc.csv', encoding='gbk',low_memory=False,index_col=None)
# link_performance_tmc['timestamp'] = pd.to_datetime(link_performance_tmc['timestamp'], format='%Y-%m-%dT%H:%M')
mapping = pd.read_csv('link_performance.csv', encoding='gbk',low_memory=False,index_col=None)


In [122]:
link_performance_tmc['timestamp']

0       2015-04-01T00:00
1       2015-04-01T00:00
2       2015-04-01T00:00
3       2015-04-01T00:00
4       2015-04-01T00:00
              ...       
1483    2015-04-01T01:55
1484    2015-04-01T01:55
1485    2015-04-01T01:55
1486    2015-04-01T01:55
1487                 NaN
Name: timestamp, Length: 1488, dtype: object

In [101]:
x = link_performance_osm.loc[[0,4]]
x=x.append(x)
x=x.append(link_performance_osm.loc[[1]])
x = x.reset_index()

In [102]:
x.drop_duplicates(['from_node_id', 'from_node_id'])

,index,name,link_id,osm_way_id,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses,timestamp,volume,travel_time,speed,notes
0,0,Leesburg Pike,18653,42713538,14870,9642,1,212.494420,3,45,NaN,trunk,2,"LINESTRING (-77.37091289999999 39.0140956, -77...",auto,2015-04-01 00:00:00,NaN,4.82942,44,NaN
24,4,Leesburg Pike,19076,59763011,12392,9992,1,292.442839,3,55,NaN,trunk,2,"LINESTRING (-77.3606994 39.0086931, -77.360374...",auto,2015-04-01 00:00:00,NaN,6.09256,48,NaN
96,1,Leesburg Pike,19078,59763021,9642,9643,1,41.060238,4,45,NaN,trunk,2,"LINESTRING (-77.36887400000001 39.0130269, -77...",auto,2015-04-01 00:00:00,NaN,0.837964,49,NaN


In [125]:
link_performance_osm = pd.DataFrame()
# link_performance_osm['name'] = None
link_performance_osm['link_id'] = None
# link_performance_osm['osm_way_id'] = None
link_performance_osm['from_node_id'] = None
link_performance_osm['to_node_id'] = None
# link_performance_osm['dir_flag'] = None
link_performance_osm['lanes'] = None
link_performance_osm['length'] = None
# link_performance_osm['free_speed'] = None
# link_performance_osm['capacity'] = None
# link_performance_osm['link_type_name'] = None
# link_performance_osm['link_type'] = None
# link_performance_osm['allowed_uses'] = None
link_performance_osm['timestamp'] = None
link_performance_osm['geometry'] = None
link_performance_osm['volume'] = None
# link_performance_osm['travel_time'] = None
link_performance_osm['speed'] = None
# link_performance_osm['notes'] = None


link_performance_osm_sub = link_performance_osm

gp = link_performance_tmc.groupby('timestamp')
for key, form in gp:
    for i in mapping.index:
        link_road_selected = link_road[(link_road['from_node_id'] == mapping.loc[i,'from_node_id']) & (link_road['to_node_id'] == mapping.loc[i,'to_node_id'])]
        trace_csv_selected = trace_csv[(trace_csv['agent_id'] == mapping.loc[i,'agent_id']) & (trace_csv['trace_id'] == mapping.loc[i,'trace_id'])]
        form_selected = form[(form['name']==trace_csv_selected['tmc'].values[0]) & (form['corridor_id']==trace_csv_selected['corridor_id'].values[0])]
        if len(form_selected)>0:
            link_performance_osm_sub = link_performance_osm_sub.append({'link_id': link_road_selected['link_id'].values[0],\
                                            'from_node_id': link_road_selected['from_node_id'].values[0],\
                                            'to_node_id': link_road_selected['to_node_id'].values[0],\
                                            'lanes': link_road_selected['lanes'].values[0],\
                                            'length': link_road_selected['length'].values[0],\
                                            'timestamp': form_selected['timestamp'].values[0],\
                                            'geometry': link_road_selected['geometry'].values[0],\
                                            'volume': form_selected['volume'].values[0],\
                                            'speed': form_selected['speed'].values[0]}, ignore_index=True)
        else:
            link_performance_osm_sub = link_performance_osm_sub.append({'link_id': link_road_selected['link_id'].values[0],\
                                            'from_node_id': link_road_selected['from_node_id'].values[0],\
                                            'to_node_id': link_road_selected['to_node_id'].values[0],\
                                            'lanes': link_road_selected['lanes'].values[0],\
                                            'length': link_road_selected['length'].values[0],\
                                            'timestamp': None,\
                                            'geometry': link_road_selected['geometry'].values[0],\
                                            'volume': None,\
                                            'speed': None}, ignore_index=True)
    link_performance_osm_sub = link_performance_osm_sub.drop_duplicates(['from_node_id', 'from_node_id']) #avoid one osm link corresponding to mutiple tmc link

    link_performance_osm = link_performance_osm.append(link_performance_osm_sub)
    link_performance_osm_sub = pd.DataFrame()
link_performance_osm.to_csv(tmc_path + os.sep +'/link_performance_osm.csv')

In [123]:
# link_performance_osm = pd.DataFrame()
# link_performance_osm['name'] = None
# link_performance_osm['link_id'] = None
# link_performance_osm['osm_way_id'] = None
# link_performance_osm['from_node_id'] = None
# link_performance_osm['to_node_id'] = None
# link_performance_osm['dir_flag'] = None
# link_performance_osm['length'] = None
# link_performance_osm['lanes'] = None
# link_performance_osm['free_speed'] = None
# link_performance_osm['capacity'] = None
# link_performance_osm['link_type_name'] = None
# link_performance_osm['link_type'] = None
# link_performance_osm['geometry'] = None
# link_performance_osm['allowed_uses'] = None
# link_performance_osm['timestamp'] = None
# link_performance_osm['volume'] = None
# link_performance_osm['travel_time'] = None
# link_performance_osm['speed'] = None
# link_performance_osm['notes'] = None


# link_performance_osm_sub = link_performance_osm

# gp = link_performance_tmc.groupby('timestamp')
# for key, form in gp:
#     for i in mapping.index:
#         link_road_selected = link_road[(link_road['from_node_id'] == mapping.loc[i,'from_node_id']) & (link_road['to_node_id'] == mapping.loc[i,'to_node_id'])]
#         trace_csv_selected = trace_csv[(trace_csv['agent_id'] == mapping.loc[i,'agent_id']) & (trace_csv['trace_id'] == mapping.loc[i,'trace_id'])]
#         form_selected = form[(form['name']==trace_csv_selected['tmc'].values[0]) & (form['corridor_id']==trace_csv_selected['corridor_id'].values[0])]
#         if len(form_selected)>0:
#             link_performance_osm_sub = link_performance_osm_sub.append({'name': link_road_selected['name'].values[0],\
#                                             'link_id': link_road_selected['link_id'].values[0],\
#                                             'osm_way_id': link_road_selected['osm_way_id'].values[0],\
#                                             'from_node_id': link_road_selected['from_node_id'].values[0],\
#                                             'to_node_id': link_road_selected['to_node_id'].values[0],\
#                                             'dir_flag': link_road_selected['dir_flag'].values[0],\
#                                             'length': link_road_selected['length'].values[0],\
#                                             'lanes': link_road_selected['lanes'].values[0],\
#                                             'free_speed': link_road_selected['free_speed'].values[0],\
#                                             'capacity': link_road_selected['capacity'].values[0],\
#                                             'link_type_name': link_road_selected['link_type_name'].values[0],\
#                                             'link_type': link_road_selected['link_type'].values[0],\
#                                             'geometry': link_road_selected['geometry'].values[0],\
#                                             'allowed_uses': link_road_selected['allowed_uses'].values[0],\
#                                             'timestamp': form_selected['timestamp'].values[0],\
#                                             'volume': form_selected['volume'].values[0],\
#                                             'travel_time': link_road_selected['length'].values[0]/form_selected['speed'].values[0],\
#                                             'speed': form_selected['speed'].values[0],\
#                                             'notes': form_selected['notes'].values[0]}, ignore_index=True)
#         else:
#             link_performance_osm_sub = link_performance_osm_sub.append({'name': link_road_selected['name'].values[0],\
#                                             'link_id': link_road_selected['link_id'].values[0],\
#                                             'osm_way_id': link_road_selected['osm_way_id'].values[0],\
#                                             'from_node_id': link_road_selected['from_node_id'].values[0],\
#                                             'to_node_id': link_road_selected['to_node_id'].values[0],\
#                                             'dir_flag': link_road_selected['dir_flag'].values[0],\
#                                             'length': link_road_selected['length'].values[0],\
#                                             'lanes': link_road_selected['lanes'].values[0],\
#                                             'free_speed': link_road_selected['free_speed'].values[0],\
#                                             'capacity': link_road_selected['capacity'].values[0],\
#                                             'link_type_name': link_road_selected['link_type_name'].values[0],\
#                                             'link_type': link_road_selected['link_type'].values[0],\
#                                             'geometry': link_road_selected['geometry'].values[0],\
#                                             'allowed_uses': link_road_selected['allowed_uses'].values[0],\
#                                             'timestamp': None,\
#                                             'volume': None,\
#                                             'travel_time': None,\
#                                             'speed': None,\
#                                             'notes': None}, ignore_index=True)
#     link_performance_osm_sub = link_performance_osm_sub.drop_duplicates(['from_node_id', 'from_node_id']) #avoid one osm link corresponding to mutiple tmc link

#     link_performance_osm = link_performance_osm.append(link_performance_osm_sub)
#     link_performance_osm_sub = pd.DataFrame()
# link_performance_osm.to_csv(tmc_path + os.sep +'/link_performance_osm.csv')

In [126]:
link_performance_osm

,link_id,from_node_id,to_node_id,lanes,length,timestamp,geometry,volume,speed
0,18653,14870,9642,3,212.494420,2015-04-01T00:00,"LINESTRING (-77.37091289999999 39.0140956, -77...",NaN,44
1,19078,9642,9643,4,41.060238,2015-04-01T00:00,"LINESTRING (-77.36887400000001 39.0130269, -77...",NaN,49
2,29356,9643,9644,3,408.118298,2015-04-01T00:00,"LINESTRING (-77.368475 39.0128263, -77.3679666...",NaN,49
3,19075,9644,12392,3,406.037382,2015-04-01T00:00,"LINESTRING (-77.36452420000001 39.0108146, -77...",NaN,48
4,19076,12392,9992,3,292.442839,2015-04-01T00:00,"LINESTRING (-77.3606994 39.0086931, -77.360374...",NaN,48
...,...,...,...,...,...,...,...,...,...
277,28010,13667,2478,4,53.275402,2015-04-01T01:55,"LINESTRING (-77.3552028 39.0059016, -77.355703...",NaN,49
278,28011,2478,2479,5,227.188989,2015-04-01T01:55,"LINESTRING (-77.3557033 39.0061814, -77.356692...",NaN,49
279,23860,2479,12397,3,336.383430,2015-04-01T01:55,"LINESTRING (-77.3578304 39.0073823, -77.357980...",NaN,49
280,23861,12397,12828,3,345.577687,2015-04-01T01:55,"LINESTRING (-77.360985 39.009155, -77.3614358 ...",NaN,49


In [24]:
import threading
import time
import ctypes
import collections
import heapq
import os.path
from sys import platform
import shutil

if platform.startswith('win32'):
    _dll_file = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'bin/MapMatching4GMNS.dll')
elif platform.startswith('linux'):
    _dll_file = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'bin/MapMatching4GMNS.so')
elif platform.startswith('darwin'):
    _dll_file = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'bin/MapMatching4GMNS.dylib')
else:
    raise Exception('Please build the shared library compatible to your OS\
                    using source files in engine_cpp!')


_cdll = ctypes.cdll.LoadLibrary(_dll_file)
# _cdll.MapMatching4GMNS.argtypes = [ctypes.c_int]

# print("To avoid complex data folder settings, please always first put the input data on the current directory.")

In [20]:
def data_cleaning():
    '''
    input_agent.csv
    agent.csv
    link_performance.csv
    '''
    # input_agent.csv
    '''
    There will be an extra comma(',') behind the geometry value in the output input_agent.csv 
    file due to different data specifications  by different programing languages. Therefore, we have to clean the data.
    '''
    if(os.path.exists("input_agent.csv")):
        f = open('input_agent.csv', encoding="utf-8")
        content = f.read()
        f.close()
        t = content.replace(",)", ")")  # delete the comma(',')
        with open("input_agent.CSV", "w", encoding='utf-8') as f1:
            f1.write(t)

    '''
    In the colab environment, if the output file already exists in the current directory, 
    an error will be reported when the newly generated output file is needed.
    '''
    # agent.csv
    if(os.path.exists("agent.csv")):
        os.remove("agent.csv")
    # link_performance.csv
    if(os.path.exists("link_performance.csv")):
        os.remove("link_performance.csv")

In [21]:
def map_match():
    '''
    input data: node.csv, link.csv and trace.csv
    output data: agent.csv and agent_performance.csv
    '''
    data_cleaning()
    print('call MapMatching4GMNS  dynamic library')
    print('\MapMatching4GMNS run starts')
    _cdll.MapMatching4GMNS()
    print('\MapMatching4GMNS run completes')

In [25]:
data_cleaning()

In [26]:
_cdll.MapMatching4GMNS()

In [8]:
from math import sin, cos, sqrt, atan2, radians
def distance_cal(lat1,lon1,lat2,lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [11]:
distance_dict = {}
# pd.DataFrame([])
for i in range(len(node)):
    node_tmc_dist = []
    for j in range(len(tmc)):
         node_tmc_dist.append(distance_cal(node['y_coord'][i],node['x_coord'][i],tmc['end_latitude'][j],tmc['end_longitude'][j]))
    distance_dict[node['node_id'][i]] = node_tmc_dist

    
df_dist=pd.DataFrame.from_dict(distance_dict)

In [38]:
# df_dist[0].
df_dist.nsmallest(5, 0)

,0,1,2,3,4,5,6,7,8,9,...,5853,5854,5855,5856,5857,5858,5859,5860,5861,5862
34,3146.511875,3153.590014,3147.343112,3147.391434,3151.437944,3151.109146,3150.900845,3150.283570,3147.641422,3149.786815,...,3149.288685,3148.065627,3150.256717,3148.797131,3149.942662,3149.928059,3150.005094,3152.691612,3149.380895,3149.054704
35,3146.797636,3153.875954,3147.628761,3147.677070,3151.723963,3151.395137,3151.186811,3150.569531,3147.927163,3150.072686,...,3149.574419,3148.351541,3150.542540,3149.083086,3150.228688,3150.214074,3150.291113,3152.977776,3149.666599,3149.340378
33,3147.198904,3154.277472,3148.029873,3148.078164,3152.125590,3151.796724,3151.588365,3150.971076,3148.328403,3150.474107,...,3149.975649,3148.753021,3150.943894,3149.484624,3150.630326,3150.615696,3150.692740,3153.379605,3150.067787,3149.741525
36,3147.666261,3154.745160,3148.497021,3148.545288,3152.593424,3152.264505,3152.056101,3151.438801,3148.795721,3150.941667,...,3150.442954,3149.220659,3151.411364,3149.952340,3151.098173,3151.083523,3151.160574,3153.847709,3150.535037,3150.208720
32,3147.666815,3154.745702,3148.497583,3148.545850,3152.593961,3152.265044,3152.056641,3151.439342,3148.796277,3150.942213,...,3150.443510,3149.221203,3151.411914,3149.952880,3151.098709,3151.084059,3151.161111,3153.848235,3150.535595,3150.209280


In [18]:
df_dist.nlargest(5, 0)

NameError: name 'df_dist' is not defined

In [52]:
distance_cal(tmc.loc[tmc['tmc']=='110-04111','end_latitude'],tmc.loc[tmc['tmc']=='110-04111','end_longitude'],tmc.loc[tmc['tmc']=='110N04103','end_latitude'],tmc.loc[tmc['tmc']=='110N04103','end_longitude'])

15.912283228698993

In [60]:
from geopy.distance import geodesic

coords_1 = (tmc.loc[tmc['tmc']=='110-04111','end_latitude'].values, tmc.loc[tmc['tmc']=='110-04111','end_longitude'].values)
coords_2 = (tmc.loc[tmc['tmc']=='110N04103','end_latitude'].values,tmc.loc[tmc['tmc']=='110N04103','end_longitude'].values)

print(geodesic(coords_1, coords_2).km)

15.899577863172917


In [17]:
import MapMatching4GMNS as mmg

To avoid complex data folder settings, please always first put the input data on the current directory.
plot4gmns, version 0.0.8
To avoid complex data folder settings, please always first put the input data on the current directory.
MapMatching4GMNS, version 0.2.8
